# Splątane sieci neuronowe CNN – architektura AlexNet

## 1. Opis
Celem projektu jest ustalenie jak różne metody głębokiego uczenia, sprawują się na danych astronomicznych. Do tego celu zostaną wykorzystane dane z projektu AstroNN [\[1\]](#1-httpsastronnreadthedocsioenstablegalaxy10sdsshtml). Projekt ten został zbudowany z danych zgromadzonych w projekcie Sloan Digital Sky Survey [\[2\]](#2-httpswwwsdss4org) (SDSS), prowadzący szczegółowy obraz nieba. Na podstawie tych danych zostały przygotowane obrazy galaktyk, które są w rozmiarze 64*64*3. Dodatkowo w projekcie są wykorzystane dane zgromadzone w projekcie Galaxy Zoo [\[3\]](#3-httpswwwzooniverseorgprojectszookeepergalaxy-zoo), gdzie dokonywana jest klasyfikacja galaktyk. Na podstawie tych danych powstały etykiety, które są wykorzystywane w projekcie.

AlexNet jest to architektura CNN z roku 2012 [\[4\]](#4-bartosz-michalski-małgorzata-plechawska-wójcik-porównanie-modeli-lenet-5-alexnet-i-googlenet-w-rozpoznawaniu-pisma-ręcznego-2022). Jest bardziej rozbudowana niż LeNet5. Została stworzona na zawody ImageNet Large Scale Visual Recognition Challenge (ILSVRC), które wygrała.

## 2. Pobranie potrzebnych bibliotek
Kolejnym krokiem jest wczytanie wszystkich potrzebnych bibliotek, dzięki którym będzie możliwe wykorzystanie ich w procesie klasyfikacji.

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPool2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam

from astroNN.datasets import galaxy10sdss
import pandas as pd
from datetime import date
import pathlib

## 3. Utworzenie daty
Następnie ustawiamy datę. Jest to potrzebne do generowania plików wynikowych. W obecnym pliku zostaną wykonane jedynie badania, a wyniki zostaną opracowane w późniejszym czasie. Ma to pozwolić zmniejszyć wymagania sprzętowe przy wykonywaniu projektu.

In [2]:
today = date.today()
day = today.strftime("%Y-%m-%d")
pathlib.Path('./Results/'+day).mkdir(parents=True, exist_ok=True)

## 4. Przygotowanie danych
W tym kroku pobieramy dane, a następnie przygotowujemy je do klasyfikacji. Modele głębokiej sieci neuronowej [\[5\]](#5-paweł-krakowiak-deep-learning-w-języku-python--konwolucyjne-sieci-neuronowe) wymaga danych z zakresu 0..1, dlatego wszystkie wartości w danych są dzielone przez 255. Powodem takiego zachowania jest fakt, że dane obrazów są przechowywane w zakresie liczb 0..255. Dzielenie przez 255 powoduje, że dane zostaną zapisane w zakresie od 0..1, zgodnie z wymaganiami modelu.

In [3]:
images, labels = galaxy10sdss.load_data()
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

features = ['Disk, Face-on, No Spiral', 'Smooth, Completely round', 'Smooth, in-between round', 'Smooth, Cigar shaped', 'Disk, Edge-on, Rounded Bulge', 'Disk, Edge-on, Boxy Bulge','Disk, Edge-on, No Bulge','Disk, Face-on, Tight Spiral', 'Disk, Face-on, Medium Spiral', 'Disk, Face-on, Loose Spiral']

x_train = x_train / 255.0
x_valid = x_valid / 255.0
x_test = x_test / 255.0

/home/grzegorz/.astroNN/datasets/Galaxy10.h5 was found!


In [4]:
reduceLR = ReduceLROnPlateau(monitor='accuracy', factor=.001, patience=1, min_delta=0.01, mode="auto")
x_train.shape, x_valid.shape, x_test.shape

((13942, 69, 69, 3), (3486, 69, 69, 3), (4357, 69, 69, 3))

## 5. Budowa modelu.
Model w tym przypadku jest bardzo prosty. Jest to model warstwowy i jako pierwsza warstwa jest to warstwa flatten. Zadaniem tej warstwy jest spłaszczenie obrazu z wymiarów 69*69 na pojedynczy ciąg, jest to warstwa wejściowa. Kolejną warstwą jest warstwa ukryta z aktywatorem RELU. Aktywator ten powoduje, że każdy otrzymany wynik ujemy, zostaje zamieniony na zero [\[6\]](#6-httpsbuiltincommachine-learningrelu-activation-function) [\[7\]](#7-httpsdatascienceeupluczenie-maszynowerelu-funkcja-aktywujaca). Pozwala to na przełamanie liniowości procesu. Ostatnią warstwą jest gęsto połączona warstwa wyjściowa. W naszym modelu klasyfikacja odbywa się dla 10 kategorii dlatego właśnie taka.

In [ ]:
model = Sequential()
model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation="relu", input_shape=(69, 69, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))
model_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=model_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 6. Uczenie
W tym momencie model zaczyna proces uczenia. Czyli otrzymuje dwa zbiory danych i etykiet. Pierwszy z nich to dane, na podstawie których model się uczy. Drugi mniejszy zbiór jest zbiorem walidacyjnym, który pozwala na sprawdzenie postępów w nauce, na danych, których model jeszcze nie widział. Pozwala to ocenić postępy w nauce już w czasie uczenia. Kolejny zbiór danych zostanie wykorzystany na końcu celem ostatecznego sprawdzenia poprawności działania modelu.

In [6]:
history = model.fit(x_train, y_train, epochs=10, callbacks=[reduceLR],validation_data=(x_valid, y_valid))

Epoch 1/10
436/436 [==============================] - 16s 35ms/step - loss: 1.3921 - accuracy: 0.4859 - val_loss: 1.6387 - val_accuracy: 0.3810 - lr: 0.0010
Epoch 2/10
436/436 [==============================] - 16s 37ms/step - loss: 1.0181 - accuracy: 0.6165 - val_loss: 1.2744 - val_accuracy: 0.6013 - lr: 0.0010
Epoch 3/10
436/436 [==============================] - 17s 38ms/step - loss: 0.9046 - accuracy: 0.6589 - val_loss: 1.2575 - val_accuracy: 0.5826 - lr: 0.0010
Epoch 4/10
436/436 [==============================] - 16s 37ms/step - loss: 0.8503 - accuracy: 0.6840 - val_loss: 1.3107 - val_accuracy: 0.5800 - lr: 0.0010
Epoch 5/10
436/436 [==============================] - 17s 39ms/step - loss: 0.8212 - accuracy: 0.6964 - val_loss: 0.8343 - val_accuracy: 0.6796 - lr: 0.0010
Epoch 6/10
436/436 [==============================] - 16s 37ms/step - loss: 0.7805 - accuracy: 0.7187 - val_loss: 2.1258 - val_accuracy: 0.3078 - lr: 0.0010
Epoch 7/10
436/436 [==============================] - 17s 

## 7. Zapis otrzymanych danych podczas nauki
Po zakończeniu uczenia zapisujemy dane, które otrzymaliśmy podczas uczenie do pliku CSV. Pozwoli nam to później przeanalizować dane w późniejszym czasie.

In [7]:
historyModelLearning = pd.DataFrame()
historyModelLearning['loss'] = history.history['loss']
historyModelLearning['accuracy'] = history.history['accuracy']
historyModelLearning['val_loss'] = history.history['val_loss']
historyModelLearning['val_accuracy'] = history.history['val_accuracy']
historyModelLearning.to_csv('./Results/'+day+'/HistoryModelLearningAlexNet-'+day+'.csv', index=True)

## 8. Sprawdzenie modelu na danych testowych. 
To jest ostateczne sprawdzenie danych. W tym sprawdzeniu otrzymamy również nie tylko informację jak jest współczynnik błędu klasyfikacji, ale również będziemy mogli przeglądać, które obrazy faktycznie zostały źle zaklasyfikowane i dzięki temu będzie możliwe poprawienie modelu.

In [8]:
predict = model.predict(x_test).argmax(axis=1)

137/137 [==============================] - 2s 12ms/step


## 9. Zapis wyników testów do pliku CSV.
To na postawie tych danych będziemy w stanie dokładniej stwierdzić, z czego wynikają problemy z klasyfikacją.

In [9]:
result = pd.DataFrame()
result['predict'] = predict
result['test'] = y_test
result.to_csv('./Results/'+day+'/ResultAlexNet-'+day+'.csv', index=False)

## Literatura

##### 1. https://astronn.readthedocs.io/en/stable/galaxy10sdss.html
##### 2. https://www.sdss4.org/
##### 3. https://www.zooniverse.org/projects/zookeeper/galaxy-zoo/
##### 4. Bartosz Michalski, Małgorzata Plechawska-Wójcik, Porównanie modeli LeNet-5, AlexNet i GoogLeNet w rozpoznawaniu pisma ręcznego, 2022
##### 5. Paweł Krakowiak, Deep learning w języku Python — Konwolucyjne Sieci Neuronowe
##### 6. https://builtin.com/machine-learning/relu-activation-function
##### 7. https://datascience.eu/pl/uczenie-maszynowe/relu-funkcja-aktywujaca/